In [ ]:
import numpy as np
import ectopylasm as ep
import ipyvolume as ipv

In [ ]:
%load_ext line_profiler

# Mock data
Let's do only 100 points here.

In [ ]:
xyz = np.array((np.random.random(100), np.random.random(100), np.random.random(100)))

# Define shape

In [ ]:
thickness = 0.2

# plane
point = (0.5, 0.5, 0.5)
normal = (0, 1, 0)  # make it normalized to one

# Filter points

In [ ]:
import sympy as sy
import tqdm

In [ ]:
def filter_points_plane_slow(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
    """
    Select the points that are within the thick plane.

    points_xyz: a vector of shape (3, N) representing N points in 3D space
    plane_point: a point in the plane
    plane_normal: the normal vector to the plane (x, y, z; any iterable)
    plane_thickness: the thickness of the plane (the distance between the two
                     composing planes)
    d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
                  specified, `plane_point` will be ignored
    """
    if d is not None:
        plane_point = ep.plane_point_from_d(plane_normal, d)
    point1, point2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)
    plane1 = sy.geometry.Plane(sy.geometry.Point3D(point1), normal_vector=plane_normal)
    plane2 = sy.geometry.Plane(sy.geometry.Point3D(point2), normal_vector=plane_normal)

    p_filtered = []
    for p_i in tqdm.tqdm(points_xyz.T):
        sy_point_i = sy.geometry.Point3D(tuple(p_i))
        if plane1.distance(sy_point_i) <= plane_thickness and plane2.distance(sy_point_i) <= plane_thickness:
            p_filtered.append(p_i)
    return p_filtered

In [ ]:
plane_points = ep.filter_points_plane(xyz, point, normal, thickness)

In [ ]:
%lprun -f filter_points_plane_slow plane_points = filter_points_plane_slow(xyz, point, normal, thickness)

This gives the following output:

```
Timer unit: 1e-06 s

Total time: 28.1696 s
File: <ipython-input-13-5c9e992f6bd9>
Function: filter_points_plane_slow at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def filter_points_plane_slow(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
     2                                               """
     3                                               Select the points that are within the thick plane.
     4                                           
     5                                               points_xyz: a vector of shape (3, N) representing N points in 3D space
     6                                               plane_point: a point in the plane
     7                                               plane_normal: the normal vector to the plane (x, y, z; any iterable)
     8                                               plane_thickness: the thickness of the plane (the distance between the two
     9                                                                composing planes)
    10                                               d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
    11                                                             specified, `plane_point` will be ignored
    12                                               """
    13         1          2.0      2.0      0.0      if d is not None:
    14                                                   plane_point = ep.plane_point_from_d(plane_normal, d)
    15         1         16.0     16.0      0.0      point1, point2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)
    16         1      17209.0  17209.0      0.1      plane1 = sy.geometry.Plane(sy.geometry.Point3D(point1), normal_vector=plane_normal)
    17         1       8052.0   8052.0      0.0      plane2 = sy.geometry.Plane(sy.geometry.Point3D(point2), normal_vector=plane_normal)
    18                                           
    19         1          1.0      1.0      0.0      p_filtered = []
    20       101      91274.0    903.7      0.3      for p_i in tqdm.tqdm(points_xyz.T):
    21       100   26006837.0 260068.4     92.3          sy_point_i = sy.geometry.Point3D(tuple(p_i))
    22       100    2046189.0  20461.9      7.3          if plane1.distance(sy_point_i) <= plane_thickness and plane2.distance(sy_point_i) <= plane_thickness:
    23        17         38.0      2.2      0.0              p_filtered.append(p_i)
    24         1          2.0      2.0      0.0      return p_filtered
```

Really surprising result! I would have thought the distance calculation would be the slowest, but in fact the Point3D construction is **ridiculously** slow! So we definitely need to get rid of this whole `sympy.geometry` thing.

In [ ]:
def plane_d(point, normal):
    """
    Calculate d factor in plane equation ax + by + cz + d = 0
    """
    return -(point[0] * normal[0] + point[1] * normal[1] + point[2] * normal[2])

In [ ]:
def point_distance_to_plane(point, plane_point, plane_normal, d=None):
    """
    Get signed distance of point to plane.
    
    The sign of the resulting distance tells you whether the point is in
    the same or the opposite direction of the plane normal vector.

    point: an iterable of length 3 representing a point in 3D space
    plane_point: a point in the plane
    plane_normal: the normal vector to the plane (x, y, z; any iterable)
    d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
                  specified, `plane_point` will be ignored
    """
    if d is None:
        d = plane_d(plane_point, plane_normal)
    
    a, b, c = plane_normal
    # from http://mathworld.wolfram.com/Point-PlaneDistance.html
    return (a * point[0] + b * point[1] + c * point[2] + d) / np.sqrt(a**2 + b**2 + c**2)

In [ ]:
def filter_points_plane_numpy(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
    """
    Select the points that are within the thick plane.

    points_xyz: a vector of shape (3, N) representing N points in 3D space
    plane_point: a point in the plane
    plane_normal: the normal vector to the plane (x, y, z; any iterable)
    plane_thickness: the thickness of the plane (the distance between the two
                     composing planes)
    d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
                  specified, `plane_point` will be ignored
    """
    if d is not None:
        plane_point = ep.plane_point_from_d(plane_normal, d)
    point1, point2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)

    p_filtered = []
    for p_i in points_xyz.T:
        distance_1 = abs(point_distance_to_plane(p_i, point1, plane_normal))
        distance_2 = abs(point_distance_to_plane(p_i, point2, plane_normal))
        if distance_1 <= plane_thickness and distance_2 <= plane_thickness:
            p_filtered.append(p_i)
    return p_filtered

In [ ]:
%timeit filter_points_plane_numpy(xyz, point, normal, thickness)

In [ ]:
%lprun -f filter_points_plane_numpy plane_points = filter_points_plane_numpy(xyz, point, normal, thickness)

This runs significantly faster. Interestingly, in a first iteration I still had tqdm in on the for loop, and that was then the dominant factor with 70% of runtime! Removing it shifted dominance to the distance functions, as we would expect:

```
Timer unit: 1e-06 s


Total time: 0.001685 s
File: <ipython-input-68-14cb67a3434b>
Function: filter_points_plane_numpy at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def filter_points_plane_numpy(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
     2                                               """
     3                                               Select the points that are within the thick plane.
     4                                           
     5                                               points_xyz: a vector of shape (3, N) representing N points in 3D space
     6                                               plane_point: a point in the plane
     7                                               plane_normal: the normal vector to the plane (x, y, z; any iterable)
     8                                               plane_thickness: the thickness of the plane (the distance between the two
     9                                                                composing planes)
    10                                               d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
    11                                                             specified, `plane_point` will be ignored
    12                                               """
    13         1          1.0      1.0      0.1      if d is not None:
    14                                                   plane_point = ep.plane_point_from_d(plane_normal, d)
    15         1         11.0     11.0      0.7      point1, point2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)
    16                                           
    17         1          0.0      0.0      0.0      p_filtered = []
    18       101         96.0      1.0      5.7      for p_i in points_xyz.T:
    19       100        759.0      7.6     45.0          distance_1 = abs(point_distance_to_plane(p_i, point1, plane_normal))
    20       100        727.0      7.3     43.1          distance_2 = abs(point_distance_to_plane(p_i, point2, plane_normal))
    21       100         77.0      0.8      4.6          if distance_1 <= plane_thickness and distance_2 <= plane_thickness:
    22        17         14.0      0.8      0.8              p_filtered.append(p_i)
    23         1          0.0      0.0      0.0      return p_filtered
    
```

So this is an increase of a factor more than ~10000 in speed! Note that this is still with profiling on, and line_profiler seems to add an overhead of a factor ~4.   

In [ ]:
28.1696 / 0.00203

In [ ]:
%lprun -f point_distance_to_plane plane_points = filter_points_plane_numpy(xyz, point, normal, thickness)

With this, we see that precalculating `d` can actually give an additional ~15% boost.

```
Timer unit: 1e-06 s

Total time: 0.004167 s
File: <ipython-input-34-3113593bd746>
Function: point_distance_to_plane at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def point_distance_to_plane(point, plane_point, plane_normal, d=None):
     2                                               """
     3                                               Get signed distance of point to plane.
     4                                               
     5                                               The sign of the resulting distance tells you whether the point is in
     6                                               the same or the opposite direction of the plane normal vector.
     7                                           
     8                                               point: an iterable of length 3 representing a point in 3D space
     9                                               plane_point: a point in the plane
    10                                               plane_normal: the normal vector to the plane (x, y, z; any iterable)
    11                                               d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
    12                                                             specified, `plane_point` will be ignored
    13                                               """
    14       200        227.0      1.1      5.4      if d is None:
    15       200        726.0      3.6     17.4          d = plane_d(plane_point, plane_normal)
    16                                               
    17       200        168.0      0.8      4.0      a, b, c = plane_normal
    18                                               # from http://mathworld.wolfram.com/Point-PlaneDistance.html
    19       200       3046.0     15.2     73.1      return (a * point[0] + b * point[1] + c * point[2] + d) / np.sqrt(a**2 + b**2 + c**2)```

In [ ]:
# def point_distance_to_plane(point, plane_point, plane_normal, d=None):
#     """
#     Get signed distance of point to plane.
    
#     The sign of the resulting distance tells you whether the point is in
#     the same or the opposite direction of the plane normal vector.

#     point: an iterable of length 3 representing a point in 3D space
#     plane_point: a point in the plane
#     plane_normal: the normal vector to the plane (x, y, z; any iterable)
#     d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
#                   specified, `plane_point` will be ignored
#     """
#     if d is None:
#         d = plane_d(plane_point, plane_normal)
    
#     a, b, c = plane_normal
#     plane_normal = np.array(plane_normal)
#     # from http://mathworld.wolfram.com/Point-PlaneDistance.html
#     return (np.sum(plane_normal * np.array(point)) + d) / np.sqrt(np.sum(plane_normal * plane_normal))

In [ ]:
# %lprun -f point_distance_to_plane plane_points = filter_points_plane_numpy(xyz, point, normal, thickness)

That increases runtime, so let's not.

One last try, precalculating d:

In [ ]:
def filter_points_plane_numpy(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
    """
    Select the points that are within the thick plane.

    points_xyz: a vector of shape (3, N) representing N points in 3D space
    plane_point: a point in the plane
    plane_normal: the normal vector to the plane (x, y, z; any iterable)
    plane_thickness: the thickness of the plane (the distance between the two
                     composing planes)
    d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
                  specified, `plane_point` will be ignored
    """
    if d is not None:
        plane_point = ep.plane_point_from_d(plane_normal, d)
    plane_point_1, plane_point_2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)
    d1 = plane_d(plane_point_1, plane_normal)
    d2 = plane_d(plane_point_2, plane_normal)

    p_filtered = []
    for p_i in points_xyz.T:
        distance_1 = point_distance_to_plane(p_i, None, plane_normal, d=d1)
        distance_2 = point_distance_to_plane(p_i, None, plane_normal, d=d2)
        if abs(distance_1) <= plane_thickness and abs(distance_2) <= plane_thickness:
            p_filtered.append(p_i)
    return p_filtered

In [ ]:
%timeit filter_points_plane_numpy(xyz, point, normal, thickness)

In [ ]:
%lprun -f filter_points_plane_numpy plane_points = filter_points_plane_numpy(xyz, point, normal, thickness)

In [ ]:
872/746

Again a small gain. In the profiling runs there's too much noise to measure the exact gain, but the timeit run shows at least a factor 1.15 faster runs.

```
Timer unit: 1e-06 s

Total time: 0.001374 s
File: <ipython-input-72-00d6dddaec0d>
Function: filter_points_plane_numpy at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def filter_points_plane_numpy(points_xyz, plane_point, plane_normal, plane_thickness, d=None):
     2                                               """
     3                                               Select the points that are within the thick plane.
     4                                           
     5                                               points_xyz: a vector of shape (3, N) representing N points in 3D space
     6                                               plane_point: a point in the plane
     7                                               plane_normal: the normal vector to the plane (x, y, z; any iterable)
     8                                               plane_thickness: the thickness of the plane (the distance between the two
     9                                                                composing planes)
    10                                               d [optional]: the constant in the plane equation ax + by + cz + d = 0; if
    11                                                             specified, `plane_point` will be ignored
    12                                               """
    13         1          2.0      2.0      0.1      if d is not None:
    14                                                   plane_point = ep.plane_point_from_d(plane_normal, d)
    15         1         12.0     12.0      0.9      plane_point_1, plane_point_2 = ep.thick_plane_points(plane_point, plane_normal, plane_thickness)
    16         1          3.0      3.0      0.2      d1 = plane_d(plane_point_1, plane_normal)
    17         1          1.0      1.0      0.1      d2 = plane_d(plane_point_2, plane_normal)
    18                                           
    19         1          0.0      0.0      0.0      p_filtered = []
    20       101         92.0      0.9      6.7      for p_i in points_xyz.T:
    21       100        595.0      6.0     43.3          distance_1 = point_distance_to_plane(p_i, None, plane_normal, d=d1)
    22       100        566.0      5.7     41.2          distance_2 = point_distance_to_plane(p_i, None, plane_normal, d=d2)
    23       100         87.0      0.9      6.3          if abs(distance_1) <= plane_thickness and abs(distance_2) <= plane_thickness:
    24        17         16.0      0.9      1.2              p_filtered.append(p_i)
    25         1          0.0      0.0      0.0      return p_filtered
```

That'll do for now. So in total, we went from:

In [ ]:
%timeit filter_points_plane_slow(xyz, point, normal, thickness)

... ~7 seconds to ~700 microseconds, i.e. a speed-up factor of 10000. Decent.

# Test

Are the results the same though?

In [ ]:
p_slow = filter_points_plane_slow(xyz, point, normal, thickness)
p_numpy = filter_points_plane_numpy(xyz, point, normal, thickness)

In [ ]:
np.array(p_slow) == np.array(p_numpy)

Yessur!